In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from scipy.fftpack import * 

## plevels 向高度场垂直插值的测试

2021.08.23

动能谱前处理部分，计算垂直平均的动能谱，在等高面上进行计算

需要针对ERA5I 和 MPAS的原始数据进行一些变量的调整，尤其是插值axis

获取hgt，得到plevels的高度，插值Ke到给定的高度层上，用作高度上Ke的垂直积分

### 数据读入

- ERA5I部分

In [2]:
# ERA5I
dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/u_98-17_daily/"
ds_u    = xr.open_dataset(dir_in + "merge_u_daily.nc") # 自动合并 Time 坐标
dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/v_98-17_daily/"
ds_v    = xr.open_dataset(dir_in + "merge_v_daily.nc") # 自动合并 Time 坐标
plevels = ds_u.level
# ds_u['uwnd'].sel( {"level":(plevels>=100) & (plevels<=400)} )
# ----- select data range -----
# lat_sel     = (ds_ke.latitude >= 10) & (ds_ke.latitude <= 60)
# lon_sel     = (ds_ke.longitude >= 70) & (ds_ke.longitude <= 140)
plevels_sel = (ds_u.level >= 0) & (ds_u.level <= 1000)
# plevels_sel = (ds_ke.plevels == 100)
# time_sel    = (ds_ke.Time.dt.year >= 1998)
# sel_dict    = {'longitude':lon_sel, "latitude":lat_sel, "plevels":plevels_sel, "Time":time_sel}
sel_dict    = {"level":plevels_sel}
uwnd = ds_u['uwnd'].isel(sel_dict)
vwnd = ds_v['vwnd'].isel(sel_dict)
ke = 0.5 * (uwnd ** 2 + vwnd ** 2)

dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/hgt/"
ds_hgt    = xr.open_dataset(dir_in + "merge_hgt_daily.nc")['hgt'] # 自动合并 Time 坐标

-   model部分

VR，RCM单独处理

In [5]:
model_type = "RCM"

# 提取动能 ke
dir_in = "/raid52/yycheng/MPAS/"+model_type+"_postprocess/"+model_type+"_merge/ke_daily_vi/"
ds_ke    = xr.open_mfdataset(dir_in + "*_"+model_type+"_ke_daily_vi.nc") # 自动合并 Time 坐标
# ----- select data range -----
# lat_sel     = (ds_ke.latitude >= 10) & (ds_ke.latitude <= 60)
# lon_sel     = (ds_ke.longitude >= 70) & (ds_ke.longitude <= 140)
plevels_sel = (ds_ke.plevels >= 10) & (ds_ke.plevels <= 400)
time_sel    = (ds_ke.Time.dt.month.isin([4,5,6,7,8]))
sel_dict    = {"plevels":plevels_sel, "Time":time_sel}
ke = ds_ke['ke'].isel(sel_dict)        # 读入选取数据到 ke_sel

# 提取等压面高度 zgird
dir_in = "/raid52/yycheng/MPAS/"+model_type+"_postprocess/"+model_type+"_merge/hgt_daily_vi/"
ds_hgt    = xr.open_mfdataset(dir_in + "*" + model_type + "_hgt_daily_vi.nc")['zgrid'].isel(sel_dict) # 自动合并 Time 坐标

In [4]:
# 注意下处理数据的维度排列
# ds_hgt
# ke

<xarray.DataArray 'hgt' (time: 3060, level: 37, lat: 160, lon: 280)>
[5072256000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1998-04-01T10:30:00 ... 2017-08-31T10:30:00
  * lon      (lon) float64 70.12 70.38 70.62 70.88 ... 139.1 139.4 139.6 139.9
  * lat      (lat) float64 15.12 15.38 15.62 15.88 ... 54.12 54.38 54.62 54.88
  * level    (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
Attributes: (12/14)
    standard_name:                 geopotential_height
    long_name:                     3-hourly Geopotential Height on Pressure L...
    units:                         m
    GridType:                      Cylindrical Equidistant Projection Grid
    datum:                         wgs84
    var_desc:                      Geopotential height
    ...                            ...
    statistic:                     Ensemble Mean
    parent_stat:                   Other
    number_of_significant_digits:  7
    ecmwf_local_table:             128
    ecmwf_parameter:               129
    actual_range:                  [ -590.3474 48486.2   ]

### 垂直插值到高度层

metpy需要带代单位进行计算，这里直接用无单位的np数组计算

In [4]:
from metpy.units import units
from metpy.interpolate import * # log_interpolate_1d
hgt_test = ds_hgt[:,:,:,:]   #['hgt']  #[:,:,:,:]
ke_test  = ke[:,:,:,:]  #[0,:,:,:]
# 等压面上的hgt高度
# iso_hgt   = units.Quantity( hgt_test.values, units('meter') )
# 想进行插值到的高度
# vi_height = units.Quantity([8500,9500,10500,11500], units('meter'))
vi_height = np.array([8500,9500,10500,11500])

ke_height = interpolate_1d(vi_height, hgt_test.values, ke_test.values, axis= 3) # 1 for ERA5I  3 for MPAS

/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/metpy/interpolate/one_dimension.py:147: UserWarning: Interpolation point out of data bounds encountered
  warnings.warn('Interpolation point out of data bounds encountered')
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/metpy/interpolate/one_dimension.py:157: RuntimeWarning: divide by zero encountered in true_divide
  var_interp = var[below] + (var[above] - var[below]) * ((x_array - xp[below])
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/metpy/interpolate/one_dimension.py:157: RuntimeWarning: invalid value encountered in multiply
  var_interp = var[below] + (var[above] - var[below]) * ((x_array - xp[below])


### 垂直积分求平均

In [5]:
vi_height = xr.DataArray(vi_height, name = "height", coords = [vi_height], dims = 'height')
# ke_height  = xr.DataArray(ke_height, coords = [ke_test.time, vi_height, ke_test.lat, ke_test.lon])
ke_height_xrdata  = xr.DataArray(ke_height, coords = [ke_test.Time, ke_test.latitude, ke_test.longitude, vi_height])
ke_vi_mean = np.trapz(ke_height_xrdata.values, vi_height, axis = 3) / (3000)
ke_vi_mean_rmnan = np.where(ke_vi_mean>1e10, -1e30, ke_vi_mean)

<ipython-input-5-252c1fd7913d>:5: RuntimeWarning: invalid value encountered in greater
  ke_vi_mean_rmnan = np.where(ke_vi_mean>1e10, -1e30, ke_vi_mean)


### 创建临时输出

-   MPAS

In [7]:
# 创建临时输出，方便后面再进行计算
dir_out = "/raid52/yycheng/MPAS/"+model_type+"_postprocess/"+model_type+"_merge/ke_daily_vi/"
# list(vr_ke_sel.coords)
coords_lon  = ke_test.longitude
coords_lat  = ke_test.latitude
coords_Time = ke_test.Time
# xr.DataArray(vr_ke_sum,dims = ['longitude', 'latitude', 'Time'])
temp_write = xr.DataArray(ke_vi_mean_rmnan, name = "vertical integration Ke", coords = [coords_Time, coords_lat ,coords_lon],attrs={"caculation":"8.5km-10.5km average Ke trapz integration"})
# temp_write.values = vr_ke_sum # 直接向values中写入
encoding_dict = {"vertical integration Ke":{"_FillValue":-1e30}}
temp_write.to_netcdf(dir_out + "ke_all_8.5-11.5_km_vi.nc",'w',\
    encoding = encoding_dict) # 此处进行NAN的处理：修改encoding方法

- ERA5I

In [ ]:
# 创建临时输出，方便后面再进行计算
dir_out = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/ke_daily_vi/"
# list(vr_ke_sel.coords)
coords_lon  = ke.lon
coords_lat  = ke.lat
coords_Time = ke.time
# xr.DataArray(vr_ke_sum,dims = ['longitude', 'latitude', 'Time'])
temp_write = xr.DataArray(ke_vi_mean_rmnan, name = "vertical integration Ke", coords = [coords_Time, coords_lat ,coords_lon],attrs={"caculation":"8.5km-10.5km average Ke trapz integration"})
# temp_write.values = vr_ke_sum # 直接向values中写入
encoding_dict = {"vertical integration Ke":{"_FillValue":-1e30}}
temp_write.to_netcdf(dir_out + "ke_all_8.5-11.5_km_vi.nc",'w',\
    encoding = encoding_dict) # 此处进行NAN的处理：修改encoding方法

## 使用metpy进行垂直插值的案例

来自： https://stackoverflow.com/questions/61034799/interpolating-gfs-winds-from-isobaric-to-height-coordinates-using-metpy

In [ ]:
from datetime import datetime
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
from metpy.interpolate import log_interpolate_1d
from siphon.catalog import TDSCatalog


gfs_url = 'https://tds.scigw.unidata.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p5deg/catalog.xml'
cat = TDSCatalog(gfs_url)

now = datetime.utcnow()

# A shortcut to NCSS
ncss = cat.datasets['Best GFS Half Degree Forecast Time Series'].subset()

query = ncss.query()
query.var = set()
query.variables('u-component_of_wind_isobaric', 'v-component_of_wind_isobaric', 'Geopotential_height_isobaric')
query.lonlat_box(north=78, south=45, east=-90, west=-220)
query.time(now)
query.accept('netcdf4')

data = ncss.get_data(query)


# Reading in the u(isobaric), v(isobaric), isobaric vars and the GPH(isobaric6) and isobaric6 vars
# These are two slightly different vertical pressure coordinates.
# We will also assign units here, and this can allow us to go ahead and convert to knots
lat = units.Quantity(data.variables['lat'][:].squeeze(), units('degrees'))
lon = units.Quantity(data.variables['lon'][:].squeeze(), units('degrees'))
iso_wind = units.Quantity(data.variables['isobaric'][:].squeeze(), units('Pa'))
iso_gph = units.Quantity(data.variables['isobaric6'][:].squeeze(), units('Pa'))
u = units.Quantity(data.variables['u-component_of_wind_isobaric'][:].squeeze(), units('m/s')).to(units('knots'))
v = units.Quantity(data.variables['v-component_of_wind_isobaric'][:].squeeze(), units('m/s')).to(units('knots'))
gph = units.Quantity(data.variables['Geopotential_height_isobaric'][:].squeeze(), units('gpm'))


# Here we will select our altitudes to interpolate onto and convert them to geopotential meters 
altitudes = ([30000., 9000., 3000.] * units('ft')).to(units('gpm'))

# Now we will interpolate the pressure coordinate for model output geopotential height to 
# estimate the pressure level for our given altitudes at each grid point
pressures_of_alts = np.zeros((len(altitudes), len(lat), len(lon)))

for ilat in range(len(lat)):
    for ilon in range(len(lon)):
        pressures_of_alts[:, ilat, ilon] = log_interpolate_1d(altitudes,
                                                              gph[:, ilat, ilon],
                                                              iso_gph)

pressures_of_alts = pressures_of_alts * units('Pa')


# Similarly, we will use our interpolated pressures to interpolate
# our u and v winds across their given pressure coordinates.
# This will provide u, v at each of our interpolated pressure
# levels corresponding to our provided initial altitudes
u_at_levs = np.zeros((len(altitudes), len(lat), len(lon)))
v_at_levs = np.zeros((len(altitudes), len(lat), len(lon)))

for ilat in range(len(lat)):
    for ilon in range(len(lon)):
        u_at_levs[:, ilat, ilon], v_at_levs[:, ilat, ilon] = log_interpolate_1d(pressures_of_alts[:, ilat, ilon],
                                                                                iso_wind,
                                                                                u[:, ilat, ilon],
                                                                                v[:, ilat, ilon])

u_at_levs = u_at_levs * units('knots')
v_at_levs = v_at_levs * units('knots')


# We can use mpcalc to calculate a wind speed array from these
wspd = mpcalc.wind_speed(u_at_levs, v_at_levs)